# 라이브러리 import

In [1]:
import os
import glob
import numpy as np
import pandas as pd


In [2]:
# 각 데이터 셋 경로
DATA_SET_PATH = 'dataset/original/'
INCOME_PATH = os.path.join(DATA_SET_PATH + 'income')
UNEMPLOYMENT_RATE_PATH = os.path.join(DATA_SET_PATH + 'unemployment_rate')
POPULATION_PATH = os.path.join(DATA_SET_PATH + 'population')
PREPROCESSED_DATA_SET_PATH = 'dataset/pre_processing/'

# 행정구역 코드정보

## 행정동코드 정보 불러오기

In [17]:
h_dng_code_df = pd.read_excel(os.path.join(POPULATION_PATH, '행정구역 코드정보.xlsx'), sheet_name='행정동코드')
h_dng_code_df.head()

,통계청행정동코드,행자부행정동코드,시도명,시군구명,행정동명
0,H_SDNG_CD,H_DNG_CD,DO_NM,CT_NM,H_DNG_NM
1,1101053,11110530,서울,종로구,사직동
2,1101054,11110540,서울,종로구,삼청동
3,1101055,11110550,서울,종로구,부암동
4,1101056,11110560,서울,종로구,평창동


In [18]:
h_dng_code_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425 entries, 0 to 424
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   통계청행정동코드  425 non-null    object
 1   행자부행정동코드  425 non-null    object
 2   시도명       425 non-null    object
 3   시군구명      425 non-null    object
 4   행정동명      425 non-null    object
dtypes: object(5)
memory usage: 16.7+ KB


## 행정동코드 데이터 전처리 

In [25]:
h_dng_code_df = h_dng_code_df.drop(0, axis=0).reset_index(drop=True)
h_dng_code_df

,통계청행정동코드,행자부행정동코드,시도명,시군구명,행정동명
0,1101053,11110530,서울,종로구,사직동
1,1101054,11110540,서울,종로구,삼청동
2,1101055,11110550,서울,종로구,부암동
3,1101056,11110560,서울,종로구,평창동
4,1101057,11110570,서울,종로구,무악동
...,...,...,...,...,...
419,1125070,11740690,서울,강동구,둔촌1동
420,1125071,11740700,서울,강동구,둔촌2동
421,1125072,11740570,서울,강동구,암사1동
422,1125073,11740610,서울,강동구,천호2동


## 전처리 행정동코드 데이터 csv 파일로 저장

In [26]:
h_dng_code_df.to_csv(os.path.join(PREPROCESSED_DATA_SET_PATH, '행정동코드.csv'), index=False)

## 유입지코드 정보 불러오기

In [4]:
resd_code_org_df = pd.read_excel(os.path.join(POPULATION_PATH, '행정구역 코드정보.xlsx'), sheet_name='유입지코드')
resd_code_org_df.head()

,RESD_CD,RESD_DO_NM,RESC_CT_NM,Unnamed: 3,Unnamed: 4
0,11110,서울,종로구,NaN,NaN
1,11140,서울,중구,NaN,NaN
2,11170,서울,용산구,NaN,NaN
3,11200,서울,성동구,NaN,NaN
4,11215,서울,광진구,NaN,NaN


In [72]:
resd_code_org_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   RESD_CD     91 non-null     int64 
 1   RESD_DO_NM  91 non-null     object
 2   RESC_CT_NM  77 non-null     object
 3   Unnamed: 3  1 non-null      object
 4   Unnamed: 4  1 non-null      object
dtypes: int64(1), object(4)
memory usage: 3.7+ KB


## 유입지코드 데이터 전처리

In [5]:
copied_resd_code_df = resd_code_org_df.copy()
copied_resd_code_df.drop(labels=['Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
copied_resd_code_df.head()

,RESD_CD,RESD_DO_NM,RESC_CT_NM
0,11110,서울,종로구
1,11140,서울,중구
2,11170,서울,용산구
3,11200,서울,성동구
4,11215,서울,광진구


In [6]:
col_rename_dict = {
  'RESD_CD':'대도시권거주지코드', 
  'RESD_DO_NM':'시도명', 
  'RESC_CT_NM':'시군구명'
}
copied_resd_code_df.rename(mapper=col_rename_dict, axis=1, inplace=True)
copied_resd_code_df.head()

,대도시권거주지코드,시도명,시군구명
0,11110,서울,종로구
1,11140,서울,중구
2,11170,서울,용산구
3,11200,서울,성동구
4,11215,서울,광진구


## 전처리 유입지코드 데이터 csv 파일로 저장

In [7]:
resd_code_df = copied_resd_code_df.copy()
resd_code_df.to_csv(os.path.join(PREPROCESSED_DATA_SET_PATH, '유입지코드.csv'), index=False)

## [TODO]

- 2018.7 남구(28170)를 미추홀구(28177)로 명칭 변경되어 데이터 파일에서 통일화 작업 필요

# 소득 데이터

- [행정동별 소득소비](https://data.seoul.go.kr/dataList/OA-22166/S/1/datasetView.do)
- 서울시 자치구별 추정 소득 및 추정 소비 정보를 제공 
- 1년중 4분기에 한번 데이터를 업데이트 하여 다음 해 1, 2, 3 분기의 값이 동일
- **행정동 코드**는 **행정안전부**에서 고시한 "주민등록 행정기관코드"를 사용하고 있습니다.
- [서울시 상권분석 서비스](https://golmok.seoul.go.kr/main.do)

## 소득 데이터 불러오기

In [55]:
income_org_df = pd.read_csv(os.path.join(INCOME_PATH, '서울시 상권분석서비스(소득소비-행정동).csv'), encoding='cp949')
income_org_df.head()

,기준_년분기_코드,행정동_코드,행정동_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,교육_지출_총금액,유흥_지출_총금액,여가_문화_지출_총금액,기타_지출_총금액,음식_지출_총금액
0,20231,11215,광진구,3192971,7,94779593000,20890784000,2984254000,2016273000,12347919000,5956030000,3158914000,4697744000,6487302000,5636812000,30603561000
1,20231,11440,마포구,3832436,8,1024107094000,30751673000,7053703000,2474929000,18507505000,3428960000,8466173000,9589375000,881500424000,8679864000,53654488000
2,20231,11530,구로구,2994059,7,1343755688000,24181621000,1395886000,2163266000,22077117000,3302443000,3962500000,3135865000,1259516797000,2954217000,21065976000
3,20231,11560,영등포구,3604887,7,211409909000,28381596000,4107783000,2787358000,21220059000,38273451000,3541679000,4888525000,36912853000,24441976000,46854629000
4,20231,11620,관악구,2771927,6,115651078000,29402491000,2679054000,1574315000,16839414000,2697308000,14762498000,5379816000,7269582000,4695170000,30351430000


In [56]:
income_org_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   기준_년분기_코드     500 non-null    int64 
 1   행정동_코드        500 non-null    int64 
 2   행정동_코드_명      500 non-null    object
 3   월_평균_소득_금액    500 non-null    int64 
 4   소득_구간_코드      500 non-null    int64 
 5   지출_총금액        500 non-null    int64 
 6   식료품_지출_총금액    500 non-null    int64 
 7   의류_신발_지출_총금액  500 non-null    int64 
 8   생활용품_지출_총금액   500 non-null    int64 
 9   의료비_지출_총금액    500 non-null    int64 
 10  교통_지출_총금액     500 non-null    int64 
 11  교육_지출_총금액     500 non-null    int64 
 12  유흥_지출_총금액     500 non-null    int64 
 13  여가_문화_지출_총금액  500 non-null    int64 
 14  기타_지출_총금액     500 non-null    int64 
 15  음식_지출_총금액     500 non-null    int64 
dtypes: int64(15), object(1)
memory usage: 62.6+ KB


## 소득 데이터 전처리

In [157]:
copied_income_org_df = income_org_df.sort_values(by='기준_년분기_코드').reset_index(drop=True)
copied_income_org_df.head()

,기준_년분기_코드,행정동_코드,행정동_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,교육_지출_총금액,유흥_지출_총금액,여가_문화_지출_총금액,기타_지출_총금액,음식_지출_총금액
0,20191,11170,용산구,4085781,8,209796299000,102207578000,1955405000,17207802000,3285343000,6669113000,1310382000,3413744000,30287570000,21355159000,22104203000
1,20191,11740,강동구,3387538,7,157294759000,15370938000,2107091000,1693041000,12954255000,2640628000,4719067000,3532185000,92891107000,3012535000,18373912000
2,20191,11680,강남구,5126808,9,1711415004000,35499680000,21985913000,13243766000,102628048000,32600818000,28804529000,14081995000,857477717000,529826857000,75265681000
3,20191,11260,중랑구,2701482,6,54397549000,16729025000,1202170000,1610451000,7216294000,2683639000,2982405000,2989841000,4419774000,2151401000,12412549000
4,20191,11200,성동구,3833236,8,60254554000,13508818000,2156687000,994529000,7058403000,5688554000,4353864000,2333434000,4112615000,3391088000,16656562000


In [158]:
date_series = copied_income_org_df['기준_년분기_코드'].astype('str')
year = date_series.apply(lambda x: x[:-1])
quarter = date_series.apply(lambda x: x[-1])

In [159]:
copied_income_org_df.insert(0, '기준년도', year.astype('int'))
copied_income_org_df.insert(1, '기준분기', quarter.astype('int'))
copied_income_org_df.drop('기준_년분기_코드', axis=1, inplace=True)

In [160]:
copied_income_org_df.head()

,기준년도,기준분기,행정동_코드,행정동_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,교육_지출_총금액,유흥_지출_총금액,여가_문화_지출_총금액,기타_지출_총금액,음식_지출_총금액
0,2019,1,11170,용산구,4085781,8,209796299000,102207578000,1955405000,17207802000,3285343000,6669113000,1310382000,3413744000,30287570000,21355159000,22104203000
1,2019,1,11740,강동구,3387538,7,157294759000,15370938000,2107091000,1693041000,12954255000,2640628000,4719067000,3532185000,92891107000,3012535000,18373912000
2,2019,1,11680,강남구,5126808,9,1711415004000,35499680000,21985913000,13243766000,102628048000,32600818000,28804529000,14081995000,857477717000,529826857000,75265681000
3,2019,1,11260,중랑구,2701482,6,54397549000,16729025000,1202170000,1610451000,7216294000,2683639000,2982405000,2989841000,4419774000,2151401000,12412549000
4,2019,1,11200,성동구,3833236,8,60254554000,13508818000,2156687000,994529000,7058403000,5688554000,4353864000,2333434000,4112615000,3391088000,16656562000


In [161]:
temp = copied_income_org_df.loc[:, '지출_총금액':] 
copied_income_org_df[temp.columns] = (temp / 3).round().astype('int')
copied_income_org_df.head()

,기준년도,기준분기,행정동_코드,행정동_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,교육_지출_총금액,유흥_지출_총금액,여가_문화_지출_총금액,기타_지출_총금액,음식_지출_총금액
0,2019,1,11170,용산구,4085781,8,69932099667,34069192667,651801667,5735934000,1095114333,2223037667,436794000,1137914667,10095856667,7118386333,7368067667
1,2019,1,11740,강동구,3387538,7,52431586333,5123646000,702363667,564347000,4318085000,880209333,1573022333,1177395000,30963702333,1004178333,6124637333
2,2019,1,11680,강남구,5126808,9,570471668000,11833226667,7328637667,4414588667,34209349333,10866939333,9601509667,4693998333,285825905667,176608952333,25088560333
3,2019,1,11260,중랑구,2701482,6,18132516333,5576341667,400723333,536817000,2405431333,894546333,994135000,996613667,1473258000,717133667,4137516333
4,2019,1,11200,성동구,3833236,8,20084851333,4502939333,718895667,331509667,2352801000,1896184667,1451288000,777811333,1370871667,1130362667,5552187333


In [162]:
income_df = pd.DataFrame(columns=copied_income_org_df.columns)

In [163]:
# 2019년도 1분기 데이터와 2018년도 1분기 데이터 같기에 2018년도 4분기 데이터 추가 작업
four_quarter_2018 = copied_income_org_df.loc[(copied_income_org_df['기준년도'] == 2019) & (copied_income_org_df['기준분기'] == 1)]
for month in range(10, 13):
  four_quarter_2018['기준년도'] = 2018
  four_quarter_2018['기준분기'] = month
  income_df = pd.concat([income_df, four_quarter_2018], ignore_index=True)

income_df.head()

/var/folders/88/5nvn4tln5150l93ty6c7l6bm0000gn/T/ipykernel_61757/454010671.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  four_quarter_2018['기준년도'] = 2018
/var/folders/88/5nvn4tln5150l93ty6c7l6bm0000gn/T/ipykernel_61757/454010671.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  four_quarter_2018['기준분기'] = month


,기준년도,기준분기,행정동_코드,행정동_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,교육_지출_총금액,유흥_지출_총금액,여가_문화_지출_총금액,기타_지출_총금액,음식_지출_총금액
0,2018,10,11170,용산구,4085781,8,69932099667,34069192667,651801667,5735934000,1095114333,2223037667,436794000,1137914667,10095856667,7118386333,7368067667
1,2018,10,11740,강동구,3387538,7,52431586333,5123646000,702363667,564347000,4318085000,880209333,1573022333,1177395000,30963702333,1004178333,6124637333
2,2018,10,11680,강남구,5126808,9,570471668000,11833226667,7328637667,4414588667,34209349333,10866939333,9601509667,4693998333,285825905667,176608952333,25088560333
3,2018,10,11260,중랑구,2701482,6,18132516333,5576341667,400723333,536817000,2405431333,894546333,994135000,996613667,1473258000,717133667,4137516333
4,2018,10,11200,성동구,3833236,8,20084851333,4502939333,718895667,331509667,2352801000,1896184667,1451288000,777811333,1370871667,1130362667,5552187333


In [164]:
for i in range(1, 5):
  quarter_df = copied_income_org_df.loc[copied_income_org_df['기준분기'] == i]
  start_month = 1 + 3 * (i - 1)
  for month in range(start_month, start_month+3):
    quarter_df['기준분기'] = month
    income_df = pd.concat([income_df, quarter_df], ignore_index=True)

income_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1575 entries, 0 to 1574
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   기준년도          1575 non-null   object
 1   기준분기          1575 non-null   object
 2   행정동_코드        1575 non-null   object
 3   행정동_코드_명      1575 non-null   object
 4   월_평균_소득_금액    1575 non-null   object
 5   소득_구간_코드      1575 non-null   object
 6   지출_총금액        1575 non-null   object
 7   식료품_지출_총금액    1575 non-null   object
 8   의류_신발_지출_총금액  1575 non-null   object
 9   생활용품_지출_총금액   1575 non-null   object
 10  의료비_지출_총금액    1575 non-null   object
 11  교통_지출_총금액     1575 non-null   object
 12  교육_지출_총금액     1575 non-null   object
 13  유흥_지출_총금액     1575 non-null   object
 14  여가_문화_지출_총금액  1575 non-null   object
 15  기타_지출_총금액     1575 non-null   object
 16  음식_지출_총금액     1575 non-null   object
dtypes: object(17)
memory usage: 209.3+ KB


/var/folders/88/5nvn4tln5150l93ty6c7l6bm0000gn/T/ipykernel_61757/3012427361.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quarter_df['기준분기'] = month


In [165]:
income_df=income_df.sort_values(by=['기준년도', '기준분기']).reset_index(drop=True)

In [166]:
col_rename_dict = {
  '기준년도':'기준년도', 
  '기준분기':'기준월', 
  '행정동_코드':'행정동_코드', 
  '행정동_코드_명':'행정동_코드_명',
  '월_평균_소득_금액':'소득_총금액',
  '소득_구간_코드':'소득_구간_코드',
  
  '지출_총금액':'지출_총금액',
  '식료품_지출_총금액':'식료품_지출_총금액',
  '의류_신발_지출_총금액':'의류_신발_지출_총금액',
  '생활용품_지출_총금액':'생활용품_지출_총금액',
  '의료비_지출_총금액':'의료비_지출_총금액',
  
  '교통_지출_총금액':'교통_지출_총금액',
  '교육_지출_총금액':'교육_지출_총금액',
  '유흥_지출_총금액':'유흥_지출_총금액',
  '여가_문화_지출_총금액':'여가_문화_지출_총금액',
  '기타_지출_총금액':'기타_지출_총금액',
  '음식_지출_총금액':'음식_지출_총금액'
}
income_df.rename(mapper=col_rename_dict, axis=1, inplace=True)
income_df.head()

,기준년도,기준월,행정동_코드,행정동_코드_명,소득_총금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,교육_지출_총금액,유흥_지출_총금액,여가_문화_지출_총금액,기타_지출_총금액,음식_지출_총금액
0,2018,10,11170,용산구,4085781,8,69932099667,34069192667,651801667,5735934000,1095114333,2223037667,436794000,1137914667,10095856667,7118386333,7368067667
1,2018,10,11740,강동구,3387538,7,52431586333,5123646000,702363667,564347000,4318085000,880209333,1573022333,1177395000,30963702333,1004178333,6124637333
2,2018,10,11680,강남구,5126808,9,570471668000,11833226667,7328637667,4414588667,34209349333,10866939333,9601509667,4693998333,285825905667,176608952333,25088560333
3,2018,10,11260,중랑구,2701482,6,18132516333,5576341667,400723333,536817000,2405431333,894546333,994135000,996613667,1473258000,717133667,4137516333
4,2018,10,11200,성동구,3833236,8,20084851333,4502939333,718895667,331509667,2352801000,1896184667,1451288000,777811333,1370871667,1130362667,5552187333


## 전처리 소득 데이터 csv 파일로 저장

In [167]:
income_df.to_csv(os.path.join(PREPROCESSED_DATA_SET_PATH, '소득소비.csv'), index=False)

# 실업률 데이터

[서울시 경제활동인구 통계](https://data.seoul.go.kr/dataList/57/S/2/datasetView.do)

1. 통계개요 
	* 통계명 : 경제활동인구 
	* 통계종류 : 서울시 15세 이상 인구중 경제활동인구 현황을 제공하는 지정ㆍ표본 통계 
	* 작성목적 : 취업, 실업 등의 경제적 특성을 조사하여 거시경제 분석과 인력자원의 개발 
	정책수립에 필요한 정보를 제공하고 정부의 고용정책 입안 및 평가에 필요한 
	기초자료를 제공하는 것을 목적으로 함 
	* 조사체계 : 지방통계청 → 통계청 
	* 공표주기 : 정기(매월) 
	* 공표범위 : 지역 - 서울시 
	내용 - 서울시 15세 이상 인구 중 경제활동 인구수 (취업, 실업) 

2. 용어설명 
	* 15세이상인구 : '15세이상인구 중 군인, 전투경찰, 공익근무요원, 형이 확정된 
	교도소 수감자' 등은 제외됨 
	* 경제활동인구 : 만 15세 이상 인구 중 취업자와 실업자를 말함 
		- 취업자 : 
			1. 조사대상 주간 중 수입을 목적으로 1시간 이상 일한 자 
			2. 자기에게 직접적으로는 이득이나 수입이 오지 않더라도 자기가구에서 
			경영하는 농장이나 사업체의 수입을 높이는 데 도운 가족종사자로서 
			주당 18시간이상 일한 자(무급가족종사자) 
			3. 직장 또는 사업체를 가지고 있으나 조사대상 주간 중 일시적인 병, 
			일기불순, 휴가 또는 연가, 노동쟁의 등의 이유로 일하지 못한 일시휴직자 
		- 실업자 : 조사대상주간에 수입 있는 일을 하지 않았고, 지난 4주간 일자리를 찾아 
		적극적으로 구직활동을 하였던 사람으로서 일자리가 주어지면 즉시 취업이 
		가능한 사람 
	* 비경제활동인구 : 조사대상 주간 중 취업자도 실업자도 아닌 만 15세 이상인 자. 
	즉 집안에서 가사와 육아를 전담하는 가정주부, 학교에 다니는 학생, 
	일을 할 수 없는 연로자와 심신 장애자, 자발적으로 자선사업이나 
	종교단체에 관여하는 자 등을 말함 

3. 기 타 
	* 구직기간 4주 기준 
	* 통계수치는 십단위에서 반올림되었으므로 전체 수치와 표내의 합계가 일치되지 않는 경우도 있을 수 있음 
	* 2015년 인구총조사(등록센서스) 결과를 토대로 소급작성된 추계인구의 변경을 반영하여 2018년 1월에 
	2000년 7월 ~ 2017년 12월까지의 자료를 변경 하였음 

4. 자 료 : 통계청「경제활동인구조사」

## 실업률 데이터 불러오기

In [60]:
unemployment_org_df = pd.read_csv(os.path.join(UNEMPLOYMENT_RATE_PATH, '서울시 경제활동인구 통계.csv'), encoding='utf-8-sig')
unemployment_org_df.head()

,시점,15세이상인구,15세이상인구.1,15세이상인구.2,경제활동인구,경제활동인구.1,경제활동인구.2,경제활동인구.3,경제활동인구.4,경제활동인구.5,경제활동인구.6,경제활동인구.7,경제활동인구.8,비경제활동인구,비경제활동인구.1,비경제활동인구.2
0,시점,소계,소계,소계,소계,소계,소계,취업자,취업자,취업자,실업자,실업자,실업자,소계,소계,소계
1,시점,합계,남자,여자,합계,남자,여자,합계,남자,여자,합계,남자,여자,합계,남자,여자
2,2018. 01,8540,4085,4456,5341,2956,2385,5105,2830,2275,237,126,110,3199,1129,2070
3,2018. 02,8538,4083,4455,5344,2946,2398,5060,2797,2263,284,149,135,3194,1137,2057
4,2018. 03,8534,4079,4455,5381,2976,2404,5083,2799,2284,298,178,121,3153,1103,2050


In [44]:
unemployment_org_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   시점         77 non-null     object
 1   15세이상인구    77 non-null     object
 2   15세이상인구.1  77 non-null     object
 3   15세이상인구.2  77 non-null     object
 4   경제활동인구     77 non-null     object
 5   경제활동인구.1   77 non-null     object
 6   경제활동인구.2   77 non-null     object
 7   경제활동인구.3   77 non-null     object
 8   경제활동인구.4   77 non-null     object
 9   경제활동인구.5   77 non-null     object
 10  경제활동인구.6   77 non-null     object
 11  경제활동인구.7   77 non-null     object
 12  경제활동인구.8   77 non-null     object
 13  비경제활동인구    77 non-null     object
 14  비경제활동인구.1  77 non-null     object
 15  비경제활동인구.2  77 non-null     object
dtypes: object(16)
memory usage: 9.8+ KB


## 실업률 데이터 전처리

In [34]:
print(unemployment_org_df.columns)
print(unemployment_org_df.loc[0].values)
print(unemployment_org_df.loc[1].values)

Index(['시점', '15세이상인구', '15세이상인구.1', '15세이상인구.2', '경제활동인구', '경제활동인구.1',
       '경제활동인구.2', '경제활동인구.3', '경제활동인구.4', '경제활동인구.5', '경제활동인구.6', '경제활동인구.7',
       '경제활동인구.8', '비경제활동인구', '비경제활동인구.1', '비경제활동인구.2'],
      dtype='object')
['시점' '소계' '소계' '소계' '소계' '소계' '소계' '취업자' '취업자' '취업자' '실업자' '실업자' '실업자'
 '소계' '소계' '소계']
['시점' '합계' '남자' '여자' '합계' '남자' '여자' '합계' '남자' '여자' '합계' '남자' '여자' '합계'
 '남자' '여자']


In [61]:
col_rename_dict = {
  '시점':'시점', 
  '15세이상인구':'15세이상인구_합계', 
  '15세이상인구.1':'15세이상인구_남자', 
  '15세이상인구.2':'15세이상인구_여자',
  '경제활동인구':'경제활동인구_합계',
  '경제활동인구.1':'경제활동인구_남자',
  '경제활동인구.2':'경제활동인구_여자',
  '경제활동인구.3':'경제활동인구_취업자_합계',
  '경제활동인구.4':'경제활동인구_취업자_남자',
  '경제활동인구.5':'경제활동인구_취업자_여자',
  '경제활동인구.6':'경제활동인구_실업자_합계',
  '경제활동인구.7':'경제활동인구_실업자_남자',
  '경제활동인구.8':'경제활동인구_실업자_여자',
  '비경제활동인구':'비경제활동인구_합계',
  '비경제활동인구.1':'비경제활동인구_남자',
  '비경제활동인구.2':'비경제활동인구_여자',
}

copied_unemployment_org_df = unemployment_org_df.copy()
copied_unemployment_org_df.rename(mapper=col_rename_dict, axis=1, inplace=True)
copied_unemployment_org_df = copied_unemployment_org_df.drop(index=[0, 1], axis=0).reset_index(drop=True)
copied_unemployment_org_df.head()

,시점,15세이상인구_합계,15세이상인구_남자,15세이상인구_여자,경제활동인구_합계,경제활동인구_남자,경제활동인구_여자,경제활동인구_취업자_합계,경제활동인구_취업자_남자,경제활동인구_취업자_여자,경제활동인구_실업자_합계,경제활동인구_실업자_남자,경제활동인구_실업자_여자,비경제활동인구_합계,비경제활동인구_남자,비경제활동인구_여자
0,2018. 01,8540,4085,4456,5341,2956,2385,5105,2830,2275,237,126,110,3199,1129,2070
1,2018. 02,8538,4083,4455,5344,2946,2398,5060,2797,2263,284,149,135,3194,1137,2057
2,2018. 03,8534,4079,4455,5381,2976,2404,5083,2799,2284,298,178,121,3153,1103,2050
3,2018. 04,8533,4078,4454,5363,2952,2412,5098,2806,2291,266,145,120,3169,1127,2043
4,2018. 05,8530,4076,4454,5381,2966,2415,5097,2801,2296,285,165,120,3149,1110,2039


In [50]:
copied_unemployment_org_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   시점             75 non-null     object
 1   15세이상인구_합계     75 non-null     object
 2   15세이상인구_남자     75 non-null     object
 3   15세이상인구_여자     75 non-null     object
 4   경제활동인구_합계      75 non-null     object
 5   경제활동인구_남자      75 non-null     object
 6   경제활동인구_여자      75 non-null     object
 7   경제활동인구_취업자_합계  75 non-null     object
 8   경제활동인구_취업자_남자  75 non-null     object
 9   경제활동인구_취업자_여자  75 non-null     object
 10  경제활동인구_실업자_합계  75 non-null     object
 11  경제활동인구_실업자_남자  75 non-null     object
 12  경제활동인구_실업자_여자  75 non-null     object
 13  비경제활동인구_합계     75 non-null     object
 14  비경제활동인구_남자     75 non-null     object
 15  비경제활동인구_여자     75 non-null     object
dtypes: object(16)
memory usage: 9.5+ KB


In [62]:
date = copied_unemployment_org_df['시점'].str.split('.')
copied_unemployment_org_df.insert(0, '기준년도', date.apply(lambda x: x[0]).astype('int'))
copied_unemployment_org_df.insert(1, '기준월', date.apply(lambda x: x[1]).astype('int'))
copied_unemployment_org_df.drop('시점', axis=1, inplace=True)

unemployment_df = copied_unemployment_org_df.copy()
unemployment_df.head()

## 전처리 실업률 데이터 csv 파일로 저장

In [64]:
unemployment_df.to_csv(os.path.join(PREPROCESSED_DATA_SET_PATH, '실업률.csv'), index=False)

# 행정동 단위 서울 생활인구 데이터

- https://data.seoul.go.kr/dataList/OA-14991/S/1/datasetView.do
- 서울시가 보유한 공공데이터와 통신데이터로 측정한 특정시점에 서울의 특정 지역에 존재하는 모든인구수 정보

## 행정동 단위 서울 생활인구 데이터 불러오기

In [67]:
# 2018 ~ 2020년까지 multi-index, 2021년부터 multi-index는 아니지만 데이터가 1 column 만큼 들여쓰기 되어 있어 '시점ID'가 index가 됨
INSEOUL_POP_PATH = os.path.join(POPULATION_PATH, '행정동 단위 서울 생활인구')
pop_org_df = pd.DataFrame()
for year in range(2018, 2025):
  inseoul_pop_file_list = sorted(glob.glob(os.path.join(INSEOUL_POP_PATH, f'LOCAL_PEOPLE_DONG_{year}*.csv')))
  
  for file_name in inseoul_pop_file_list:
    if ("201811" in file_name) or ("201910" in file_name):
      continue
    df = pd.read_csv(file_name, encoding='utf-8-sig')
    org_col_name = df.columns
    df = df.dropna(axis=1).reset_index()
    df.columns = org_col_name

    pop_org_df = pd.concat([pop_org_df, df])

In [68]:
pop_org_df

,기준일ID,시간대구분,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20180101,0,11110515,14471.4116,527.7090,336.2573,381.2662,551.7157,567.6465,486.6423,...,389.1351,473.6825,635.6879,729.5227,610.9207,575.3451,494.6534,463.5910,449.7000,1351.0037
1,20180101,0,11110530,23059.7992,715.9096,318.1821,461.8743,1074.8617,959.5067,922.5332,...,1055.1869,975.7669,1025.8741,957.0444,903.3613,778.7224,774.6881,666.7125,453.2184,2046.0369
2,20180101,0,11110540,4928.6410,83.2518,63.7905,86.3778,255.9185,253.8867,188.0076,...,172.6411,142.7509,191.2020,197.6714,189.4098,168.4414,223.6928,191.1262,128.7450,491.3451
3,20180101,0,11110550,14060.9810,268.6168,208.2819,331.6045,483.0823,421.1730,331.6272,...,384.2033,413.4419,545.6471,511.8307,701.3981,679.1825,571.2549,629.6860,462.4967,1543.6116
4,20180101,0,11110560,20322.3298,515.5184,372.8914,552.9593,594.7631,559.0723,472.6954,...,513.4737,568.1072,711.6436,706.7804,892.6337,969.5679,982.7466,872.2911,658.0842,2322.3739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315451,20240331,23,11110560,18940.5621,531.9950,384.8094,576.3290,660.8280,424.7776,381.3306,...,456.1228,460.0157,673.1688,721.6437,865.3681,827.9273,1027.6007,857.8810,639.2480,1675.1834
315452,20240331,23,11110670,8464.5677,55.4443,38.5192,92.0450,232.7742,291.0509,348.6730,...,318.4142,278.9115,353.5779,415.6141,491.8446,482.3313,428.5704,377.8645,272.6651,518.5588
315453,20240331,23,11740690,4918.5119,184.4127,139.7659,354.7135,114.4637,141.6816,120.7297,...,108.7438,116.5319,165.2712,206.6911,260.4003,181.6257,202.1399,176.4466,139.2120,367.4747
315454,20240331,23,11560720,24933.7367,369.5424,197.5816,437.0123,595.4823,875.1765,990.5305,...,888.0321,955.6183,1137.9109,865.9882,1134.2272,1093.0010,1255.0108,1125.4076,769.5949,1542.7417


## 행정동 단위 서울 생활인구 데이터 csv 파일로 저장하기

In [69]:
pop_org_df.to_csv(os.path.join(PREPROCESSED_DATA_SET_PATH, '유동인구.csv'), index=False)

## 이슈있는 201811, 201910 데이터 처리

In [10]:
file_name_list = ['201811', '201910']
file_encoding_list = ['utf-8-sig', 'cp949']

issue_data_df = pd.DataFrame()

for i in range(0, 2):
  file_name = f'LOCAL_PEOPLE_DONG_{file_name_list[i]}.csv'
  df = pd.read_csv(os.path.join(INSEOUL_POP_PATH, file_name), encoding=file_encoding_list[i])

  issue_data_df = pd.concat([issue_data_df, df])

issue_data_df

/var/folders/88/5nvn4tln5150l93ty6c7l6bm0000gn/T/ipykernel_91376/3676849776.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(INSEOUL_POP_PATH, file_name), encoding=file_encoding_list[i])


,기준일ID,시간대구분,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20181101,0,11110515,14799.0314,489.8446,312.1301,493.4818,484.2561,463.3706,486.0634,...,442.1121,483.7973,694.3211,768.4976,803.8630,572.8530,476.6775,407.1267,491.9050,1274.6222
1,20181101,0,11110540,5072.4639,68.5522,52.5271,331.0671,252.8436,172.7127,179.7855,...,122.8208,142.8454,200.8265,185.8934,155.6270,186.6237,198.3063,185.9573,171.8843,503.2241
2,20181101,0,11110550,13510.7258,296.9959,230.2863,355.1712,530.7080,438.1504,361.4088,...,382.5184,437.4985,541.8144,493.2076,642.4960,655.1849,488.8321,612.6179,323.6644,1417.2739
3,20181101,0,11110560,22047.6664,631.9267,457.0936,632.7882,612.5082,608.7870,513.3366,...,467.2121,570.1048,795.0337,781.8997,1067.2416,932.3145,1074.0614,1025.2258,747.4158,2495.0940
4,20181101,0,11170530,10764.3579,210.1115,101.3827,195.1805,562.6486,550.6451,562.9493,...,544.9739,479.1962,453.2349,330.4576,335.1968,313.0150,367.0382,299.8554,234.3563,519.1046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183163,20191031,23,11350612,17632.1219,681.3252,602.3885,675.6099,478.1793,414.3026,390.6870,...,580.3386,562.1668,618.5708,786.2821,929.2053,729.1042,817.4113,633.5907,463.0302,1213.9416
183164,20191031,23,11170630,29172.2717,1316.3336,706.9839,578.3604,535.9633,575.9288,813.9943,...,790.0160,1031.9463,1481.3725,1471.4765,1398.3573,1094.3471,1141.3413,1059.4292,694.9434,2446.4439
183165,20191031,23,11545710,14687.8462,456.6597,265.8582,302.8503,381.7735,539.3950,520.8559,...,483.2572,466.8911,587.2631,506.3171,629.7512,619.3778,638.9704,586.7954,580.1744,1081.0983
183166,20191031,23,11320522,19245.5754,958.9530,418.9691,349.7508,388.4940,524.5519,509.1088,...,588.6962,600.6033,697.4738,736.8514,755.1141,677.4285,857.4224,869.1521,713.8687,1663.5754


In [18]:
pop_df = pd.concat([pop_org_df, issue_data_df])
pop_df.sort_values(by=['기준일ID', '시간대구분'], inplace=True)

In [26]:
pop_df.to_csv(os.path.join(PREPROCESSED_DATA_SET_PATH, '유동인구_2.csv'), index=False)